# Clifford+RZ

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

In [2]:
import math
import os
import random

import numpy as np
import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc
from qiskit.compiler import transpile
from pyqrack import QrackSimulator

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ['QRACK_MAX_CPU_QB']='-1'

In [3]:
width = 6
max_magic = 6

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [4]:
sqrt1_2 = 1 / math.sqrt(2)

def x_to_y(circ, q):
    circ.s(q)
    return 1

def x_to_z(circ, q):
    circ.h(q)
    return 1

def y_to_z(circ, q):
    circ.adjs(q)
    circ.h(q)
    return 2

def y_to_x(circ, q):
    circ.adjs(q)
    return 1

def z_to_x(circ, q):
    circ.h(q)
    return 1

def z_to_y(circ, q):
    circ.h(q)
    circ.s(q)
    return 2

def cx(circ, q1, q2):
    circ.mcx([q1], q2)
    return 1

def cy(circ, q1, q2):
    circ.mcy([q1], q2)
    return 1

def cz(circ, q1, q2):
    circ.mcz([q1], q2)
    return 1

def acx(circ, q1, q2):
    circ.macx([q1], q2)
    return 1

def acy(circ, q1, q2):
    circ.macy([q1], q2)
    return 1

def acz(circ, q1, q2):
    circ.macz([q1], q2)
    return 1

def swap(circ, q1, q2):
    circ.swap(q1, q2)
    return 1

def nswap(circ, q1, q2):
    circ.mcz([q1], q2)
    circ.swap(q1, q2)
    circ.mcz([q1], q2)
    return 3

def pswap(circ, q1, q2):
    circ.mcz([q1], q2)
    circ.swap(q1, q2)
    return 2

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.mcz([q1], q2)
    return 2

def iswap(circ, q1, q2):
    circ.iswap(q1, q2)
    return 1

def iiswap(circ, q1, q2):
    circ.adjiswap(q1, q2)
    return 1

def random_circuit(width, circ):
    t_count = 0
    gate_count = 0
    bit_depths = width * [0]

    single_bit_gates = { 0: (z_to_x, z_to_y), 1: (x_to_y, x_to_z), 2: (y_to_z, y_to_x) } 
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz

    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))

    # Don't repeat bases:
    bases = [0] * width
    directions = [0] * width

    for i in range(3 * width):
        # Single bit gates
        for j in range(width):
            # Reset basis, every third layer
            if i % 3 == 0:
                bases[j] = random.randint(0, 2)
                directions[j] = random.randint(0, 1)

            # Sequential basis switch
            gate = single_bit_gates[bases[j]][directions[j]]
            g_count = gate(circ, j)
            gate_count += g_count
            bit_depths[j] += g_count

            # Cycle through all 3 Pauli axes, every 3 layers
            if directions[j]:
                bases[j] -= 1
                if bases[j] < 0:
                    bases[j] += 3
            else:
                bases[j] += 1
                if bases[j] > 2:
                    bases[j] -= 3

            # Rotate around local Z axis
            rnd = random.randint(0, 3)
            if rnd == 0:
                circ.s(j)
            elif rnd == 1:
                circ.z(j)
            elif rnd == 2:
                circ.adjs(j)
            # else - identity
            if rnd < 3:
                gate_count += 1
                bit_depths[j] += 1

            if (t_count < max_magic) and (width * width * random.random() / max_magic) < 1:
                circ.u(j, 0, random.uniform(0, 4 * math.pi), 0)
                gate_count += 1
                bit_depths[j] += 1
                t_count += 1

        # Nearest-neighbor couplers:
        ############################
        # gate = gateSequence.pop(0)
        # gateSequence.append(gate)
        # for row in range(1, row_len, 2):
        #     for col in range(row_len):
        #         temp_row = row
        #         temp_col = col
        #         temp_row = temp_row + (1 if (gate & 2) else -1);
        #         temp_col = temp_col + (1 if (gate & 1) else 0)

        #         if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
        #             continue

        #         b1 = row * row_len + col
        #         b2 = temp_row * row_len + temp_col

        #         if (b1 >= width) or (b2 >= width):
        #             continue

        #         g = random.choice(two_bit_gates)
        #         g_count = g(circ, b1, b2)
        #         gate_count += g_count
        #         bit_depths[b1] += g_count
        #         bit_depths[b2] += g_count

        # Fully-connected couplers:
        ###########################
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            g = random.choice(two_bit_gates)
            g_count = g(circ, b1, b2)
            gate_count += g_count
            bit_depths[b1] += g_count
            bit_depths[b2] += g_count

    print("Gate count (before optimization): ", gate_count)

    return circ

If we have Qiskit and numpy installed, we can convert to a Qiskit circuit. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In [5]:
qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False)
random_circuit(width, qsim)
qsim.out_to_file('qrack_circuit.chp')
qsim.reset_all()
qsim = None

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Gate count (before optimization):  297


In [6]:
circ = QrackSimulator.file_to_qiskit_circuit('qrack_circuit.chp')

print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

      ┌───┐┌───┐                             ┌───┐                             »
 q_0: ┤ S ├┤ H ├───────────────────────■─────┤ S ├─────────────────────────────»
      ├───┤└───┘        ┌───┐          │     └───┘                             »
 q_1: ┤ H ├───────■─────┤ H ├──────────┼───────────────────────────────────────»
      ├───┤┌───┐  │     └───┘          │     ┌───┐   ┌───┐                     »
 q_2: ┤ S ├┤ H ├──┼───────────────■────┼─────┤ H ├───┤ S ├─────────────────────»
      ├───┤└───┘  │               │    │     └───┘   └───┘┌───┐                »
 q_3: ┤ H ├───────┼───────────────┼────┼───────────────■──┤ H ├─────────────■──»
      └───┘       │               │    │               │  └───┘   ┌───┐     │  »
 q_4: ────────────┼───────────────┼────┼───────────────┼────■─────┤ S ├─────┼──»
      ┌───┐┌───┐  │               │    │               │    │     └───┘     │  »
 q_5: ┤ S ├┤ H ├──┼───────■───────┼────┼───────────────┼────┼───────────────┼──»
      └───┘└───┘  │     ┌─┴─

We can further optimize by post-selecting with tensor network software.

In [7]:
tc.set_backend("tensorflow")
tc.set_contractor("auto")
tc.set_dtype("complex64")

net = tc.Circuit.from_qiskit(circ)
for b in range(width, circ.width()):
    net.post_select(b, keep=0)
net = tcsc.simple_compile(net)[0]

We can sample with `tensorcircuit` at this point, or we can try another round of optimization.

In [8]:
# %%time
# print(net.sample(allow_state=True, batch=1024, format="count_dict_bin"))

We can convert back to a Qiskit circuit.

In [9]:
circ = net.to_qiskit()
print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

omit non unitary gate in tensorcircuit when transforming to qiskit: any
omit non unitary gate in tensorcircuit when transforming to qiskit: any
omit non unitary gate in tensorcircuit when transforming to qiskit: any


      ┌───┐┌───┐                       ┌───┐                                   »
 q_0: ┤ S ├┤ H ├───────────────────■───┤ S ├───────────────────────────────────»
      ├───┤└───┘      ┌───┐        │   └───┘                                   »
 q_1: ┤ H ├───────■───┤ H ├────────┼───────────────────────────────────────────»
      ├───┤┌───┐  │   └───┘        │   ┌───┐ ┌───┐                             »
 q_2: ┤ S ├┤ H ├──┼───────────■────┼───┤ H ├─┤ S ├─────────────────────────────»
      ├───┤└───┘  │           │    │   └───┘ └───┘┌───┐             ┌───┐ ┌───┐»
 q_3: ┤ H ├───────┼───────────┼────┼───────────■──┤ H ├─────────■───┤ H ├─┤ S ├»
      └───┘       │           │    │           │  └───┘ ┌───┐   │   ├───┤ └───┘»
 q_4: ────────────┼───────────┼────┼───────────┼────■───┤ S ├───┼───┤ H ├───■──»
      ┌───┐┌───┐  │           │    │           │    │   └───┘   │   └───┘   │  »
 q_5: ┤ S ├┤ H ├──┼─────■─────┼────┼───────────┼────┼───────────┼───────────┼──»
      └───┘└───┘  │   ┌─┴─┐ 

Some phase gates tend to occur at the beginning of the circuit: these can safely be ignored. In fact, we can trace out the evolution of separable single-qubit states from the beginning of the circuit until they become entangled, for more gate elimination. Deterministic control qubits can be eliminated, as well. Further, any ancilla that doesn't act as a control qubit can be safely dropped.

In [10]:
to_cut = []
state = circ.width() * [False]
basis = circ.width() * [False]
has_depth = circ.width() * [False]
has_control = circ.width() * [False]
for i in range(len(circ.data)):
    op = circ.data[i].operation
    qubits = circ.data[i].qubits
    first_qubit = circ.find_bit(qubits[0])[0]
    if has_depth[first_qubit]:
        if (len(qubits) == 2) and op.name != "swap":
            qc = circ.find_bit(qubits[0])[0]
            qt = circ.find_bit(qubits[1])[0]
            has_control[qc] = True
            has_control[qt] = True
        continue
    if len(qubits) == 1:
        if op.name == "h":
            basis[first_qubit] = not basis[first_qubit]
        elif ((not basis[first_qubit]) and ((op.name == "z") or (op.name == "s") or (op.name == "sdg"))) or (basis[first_qubit] and (op.name == "x")):
            to_cut.append(i)
        elif ((not basis[first_qubit]) and ((op.name == "x") or (op.name == "y"))) or (basis[first_qubit] and ((op.name == "y") or (op.name == "z"))):
            state[first_qubit] = not state[first_qubit]
        else:
            has_depth[first_qubit] = True
    elif op.name == "swap":
        second_qubit = has_depth[circ.find_bit(qubits[1])[0]]
        has_depth[first_qubit], has_depth[second_qubit] = has_depth[second_qubit], has_depth[first_qubit]
        has_control[first_qubit], has_control[second_qubit] = has_control[second_qubit], has_control[first_qubit]
        state[first_qubit], state[second_qubit] = state[second_qubit], state[first_qubit]
        basis[first_qubit], basis[second_qubit] = basis[second_qubit], basis[first_qubit]
        if (not has_depth[first_qubit]) and (not has_depth[second_qubit]) and (state[first_qubit] == state[second_qubit]) and (basis[first_qubit] == basis[second_qubit]):
            to_cut.append(i)
    elif not basis[first_qubit]:
        if state[first_qubit]:
            circ.data[i].operation.name = "x"
            del circ.data[i].qubits[0]
            i -= 1
        else:
            to_cut.append(i)
    else:
        qc = circ.find_bit(qubits[0])[0]
        qt = circ.find_bit(qubits[1])[0]
        has_control[qc] = True
        has_control[qt] = True
        has_depth[qc] = True
        has_depth[qt] = True

to_cut.reverse()
for i in to_cut:
    del circ.data[i]

for i in range(len(has_control)):
    c = has_control[i]
    if c or (i < width):
        continue

    to_cut = []
    for j in range(len(circ.data)):
        for q in circ.data[j].qubits:
            if i == circ.find_bit(q)[0]:
                to_cut.append(j)

    to_cut.reverse()
    for i in to_cut:
        del circ.data[i]
    
print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

      ┌───┐               ┌───┐                                    ┌───┐       »
 q_0: ┤ H ├────────────■──┤ S ├────────────────────────────────────┤ X ├───────»
      ├───┤            │  └───┘ ┌───┐                              └─┬─┘       »
 q_1: ┤ H ├────────────┼────■───┤ H ├────────────────────────────────┼─────────»
      ├───┤            │    │   ├───┤ ┌───┐                          │         »
 q_2: ┤ H ├───────■────┼────┼───┤ H ├─┤ S ├──────────────────────────┼─────────»
      ├───┤       │    │    │   └───┘ └───┘ ┌───┐       ┌───┐ ┌───┐  │         »
 q_3: ┤ H ├───────┼────┼────┼───────────■───┤ H ├───■───┤ H ├─┤ S ├──■─────────»
      ├───┤       │    │    │           │   └───┘   │   └───┘ └───┘┌───┐       »
 q_4: ┤ H ├───────┼────┼────┼───────────┼───────────┼───────────■──┤ S ├───────»
      ├───┤       │    │    │           │           │           │  └───┘ ┌───┐ »
 q_5: ┤ H ├──■────┼────┼────┼───────────┼───────────┼───────────┼────■───┤ S ├─»
      └───┘┌─┴─┐  │    │    

If any ancilla qubit is nothing but an injection gadget, we can re-inject it.

In [11]:
sqrt1_2 = 1 / math.sqrt(2)
passable_gates = ["unitary", "x", "y", "z", "s", "sdg"]

unitary_stub = ""
for inst in circ.data:
    if inst.operation.name != "unitary":
        continue
    unitary_stub = inst
    break

if unitary_stub != "":
    for i in range(width, circ.width()):
        non_clifford = np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]])
        to_cut = []
        for j in reversed(range(len(circ.data))):
            id = circ.find_bit(qubits[0])[0]
            if id == i:
                unitary_stub.qubits = (qubits[0],)
                break
        for j in reversed(range(len(circ.data))):
            op = circ.data[j].operation
            qubits = circ.data[j].qubits

            id = circ.find_bit(qubits[0])[0]
            if (len(qubits) < 2) and (id == i):
                to_cut.append(j)
                if op.name == "unitary":
                    non_clifford = np.matmul(non_clifford, op.params[0])
                elif op.name == "h":
                    non_clifford = np.matmul(non_clifford, np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]]))
                elif op.name == "x":
                    non_clifford = np.matmul(non_clifford, np.array([[0, 1], [1, 0]]))
                elif op.name == "y":
                    non_clifford = np.matmul(non_clifford, np.array([[0, -1j], [1j, 0]]))
                elif op.name == "z":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1]]))
                elif op.name == "s":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, 1j]]))
                elif op.name == "sdg":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1j]]))
                else: 
                    print("Something went wrong!")

                continue

            if len(qubits) < 2:
                continue

            if (id == i) or not (op.name == "cx"):
                non_clifford = np.matmul(np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]]), non_clifford)
                unitary_stub.operation.params = [np.asarray(non_clifford)]
                circ.data.insert(j + 1, unitary_stub)
                break

            if (circ.find_bit(qubits[1])[0] == i) and (op.name == "cx"):
                circ.data[j].operation.name = "unitary"
                circ.data[j].operation.params.append(non_clifford)
                circ.data[j].qubits = (qubits[0],)
        
        for i in to_cut:
            del circ.data[i]

print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

      ┌───┐               ┌───┐                                   ┌───┐      »
 q_0: ┤ H ├────────────■──┤ S ├───────────────────────────────────┤ X ├──────»
      ├───┤            │  └───┘┌───┐                              └─┬─┘      »
 q_1: ┤ H ├────────────┼────■──┤ H ├────────────────────────────────┼────────»
      ├───┤            │    │  ├───┤┌───┐                           │        »
 q_2: ┤ H ├───────■────┼────┼──┤ H ├┤ S ├───────────────────────────┼────────»
      ├───┤       │    │    │  └───┘└───┘ ┌───┐       ┌───┐ ┌───┐   │        »
 q_3: ┤ H ├───────┼────┼────┼─────────■───┤ H ├───■───┤ H ├─┤ S ├───■────────»
      ├───┤       │    │    │         │   └───┘   │   └───┘ └───┘ ┌───┐      »
 q_4: ┤ H ├───────┼────┼────┼─────────┼───────────┼───────────■───┤ S ├──────»
      ├───┤       │    │    │         │           │           │   └───┘      »
 q_5: ┤ H ├──■────┼────┼────┼─────────┼───────────┼───────────┼───────────■──»
      └───┘┌─┴─┐  │    │    │  ┌───┐  │  ┌─────┐  │ 

`QrackSimulator` can run a Qiskit circuit.

In [12]:
# basis_gates=["rz", "h", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap"]
# circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)

os.rename('qrack_circuit.chp', 'qrack_circuit_original.chp')

qsim = QrackSimulator(circ.width(), isSchmidtDecomposeMulti=False, isSchmidtDecompose=False)
qsim.run_qiskit_circuit(circ, 0)

qsim.out_to_file("qrack_circuit.chp")

lines = []
with open('qrack_circuit.chp','r',encoding='utf-8') as file:
    lines = file.readlines()
lines[0] = str(width) + "\n"
with open('qrack_circuit.chp', 'w', encoding='utf-8') as file:
    file.writelines(lines)

We can also sample with Qrack, as opposed to tensor circuits.

In [13]:
lines = []
with open('qrack_circuit.chp','r',encoding='utf-8') as file:
    lines = file.readlines()
chp_qb_count = int(lines[1])
is_clifford = True
for i in range(2 + 2 * chp_qb_count, len(lines)):
    if lines[i] != "(1,0) (0,0) (0,0) (1,0)\n":
        is_clifford = False
        break

if is_clifford:
    qsim = QrackSimulator.in_from_file("qrack_circuit.chp")
    %time
    print(qsim.measure_shots(list(range(width)), 1024))

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 6.91 µs
[18, 26, 48, 33, 7, 34, 37, 7, 34, 42, 34, 48, 21, 52, 52, 56, 4, 56, 41, 51, 25, 42, 51, 4, 0, 7, 7, 25, 48, 30, 37, 30, 18, 60, 29, 30, 21, 55, 37, 8, 30, 48, 3, 37, 8, 22, 63, 15, 29, 56, 41, 3, 18, 8, 30, 4, 34, 8, 46, 26, 4, 60, 3, 15, 33, 38, 46, 17, 51, 34, 46, 8, 17, 41, 37, 4, 26, 25, 41, 15, 48, 33, 45, 51, 38, 51, 37, 21, 38, 0, 17, 4, 21, 45, 48, 41, 7, 42, 3, 48, 17, 11, 45, 22, 18, 56, 38, 63, 48, 52, 42, 59, 0, 55, 63, 51, 4, 38, 55, 26, 42, 21, 18, 0, 63, 11, 30, 7, 42, 52, 0, 34, 51, 0, 8, 15, 17, 38, 59, 11, 29, 48, 59, 33, 51, 45, 34, 17, 42, 33, 0, 21, 60, 3, 8, 4, 63, 26, 52, 18, 34, 33, 8, 42, 37, 33, 37, 46, 33, 26, 34, 22, 11, 60, 8, 21, 46, 33, 48, 45, 55, 33, 0, 42, 46, 59, 22, 60, 41, 26, 46, 8, 55, 33, 18, 8, 51, 59, 18, 56, 30, 33, 21, 33, 41, 45, 33, 46, 51, 15, 11, 3, 18, 51, 4, 30, 56, 4, 22, 4, 8, 3, 41, 25, 55, 21, 52, 52, 30, 21, 0, 4, 4, 29, 46, 41, 18, 0, 17, 21, 37, 38, 21, 12, 42, 25,